In [1]:
import pandas as pd
import numpy as np
from finance_byu import fama_macbeth
import warnings
warnings.filterwarnings("ignore")

In [2]:
ab_pos = pd.read_csv('Final Variables/abnormal_positive_ratio.csv', index_col=0)
ab_neg = pd.read_csv('Final Variables/abnormal_negative_ratio.csv', index_col=0)
pos = pd.read_csv('Final Variables/positive_ratio.csv', index_col=0)
neg = pd.read_csv('Final Variables/negative_ratio.csv', index_col=0)

variables = ['NR', 'ABNR', 'PR', 'ABPR']
ab_pos_lag = ab_pos[ab_pos.index <= '2022-11']
ab_pos_lead = ab_pos.iloc[1:(len(ab_pos_lag)+1)]

ab_neg_lag = ab_neg[ab_neg.index <= '2022-11']
ab_neg_lead = ab_neg.iloc[1:(len(ab_neg_lag)+1)]

pos_lag = pos[pos.index <= '2022-11']
pos_lead = pos.iloc[1:(len(pos_lag)+1)]

neg_lag = neg[neg.index <= '2022-11']
neg_lead = neg.iloc[1:(len(neg_lag)+1)]

ab_pos_means = ab_pos_lag.mean()
ab_neg_means = ab_neg_lag.mean()
pos_means = pos_lag.mean()
neg_means = neg_lag.mean()

def summary_stats(df):
    
    column_stats = pd.DataFrame([df.mean(), df.std(), df.skew(), df.kurt(), df.min(), df.quantile(0.01), df.quantile(0.05), df.quantile(0.1), df.quantile(0.25), df.quantile(0.5), df.quantile(0.75), df.quantile(0.9), df.quantile(0.95), df.quantile(0.99), df.max()])
    column_stats.index = ['Mean', 'SD', 'Skew', 'Kurt', 'Min', '1%', '5%', '10%', '25%', 'Median', '75%', '90%', '95%', '99%', 'Max']
    
    return column_stats

summary_stats = pd.concat([summary_stats(neg_means), summary_stats(ab_neg_means), summary_stats(pos_means), summary_stats(ab_pos_means)], keys = variables, axis=1).transpose()
summary_stats.index = summary_stats.index.droplevel(1)
summary_stats.to_csv('summary statistic/table1.csv')

ab_pos_means_df = pd.DataFrame(ab_pos_means, columns=['ABPR'])
ab_neg_means_df = pd.DataFrame(ab_neg_means, columns=['ABNR'])
pos_means_df = pd.DataFrame(pos_means, columns=['PR'])
neg_means_df = pd.DataFrame(neg_means, columns=['NR'])

lead_ab_pos_means_df = pd.DataFrame(ab_pos_lead.mean(), columns=['Lead_ABPR'])
lead_ab_neg_means_df = pd.DataFrame(ab_neg_lead.mean(), columns=['Lead_ABNR'])
lead_pos_means_df = pd.DataFrame(pos_lead.mean(), columns=['Lead_R'])
lead_neg_means_df = pd.DataFrame(neg_lead.mean(), columns=['Lead_NR'])

data_df = pd.concat([neg_means_df, ab_neg_means_df, pos_means_df, ab_pos_means_df, lead_neg_means_df, lead_ab_neg_means_df, lead_pos_means_df, lead_ab_pos_means_df], axis=1)
correlation_matrix = data_df.corr()
correlation_matrix.to_csv('summary statistic/table2.csv')

In [3]:
def fama(data, shift_factor, control = []):

    future = data.shift(shift_factor)
    
    comb = []
    tickers = data.columns

    for i in range(len(tickers)):
        data[tickers[i]].index = pd.to_datetime(data[tickers[i]].index)
        combined = pd.concat([data[tickers[i]], future[tickers[i]].rename('future')], axis = 1)
        if control != []:
            combined = pd.concat([combined, control[i]], axis = 1)

        comb.append(combined[:shift_factor])

    for i in range(len(comb)):
        comb[i] = comb[i].reset_index()
        comb[i]['index'] = pd.to_datetime(pd.to_datetime(comb[i]['index']).dt.strftime('%Y-%m'))

    results = []
    tickers = data.columns        

    for i in range(len(comb)):
        xvar = [tickers[i]]
        comb[i] = comb[i].dropna(axis = 1, how = 'all')
        comb[i] = comb[i].dropna(axis = 0, how = 'any')
        if control != []:
            for j in range(3, len(comb[i].columns)):
                xvar.append(comb[i].columns[j])
        comb[i] = comb[i].dropna(axis = 1, how = 'all')
        comb[i] = comb[i].dropna(axis = 0, how = 'any')
        results.append(fama_macbeth.fama_macbeth_master(comb[i], t = 'index', yvar = 'future', xvar = xvar, intercept=True))

    alpha = []
    beta = []
    for i in range(len(results)):
        alpha.append(results[i]['intercept'])
        beta.append(results[i][tickers[i]])

    return alpha, beta, results

In [4]:
ab_neg.index = pd.to_datetime(ab_neg.index)
ab_pos.index = pd.to_datetime(ab_pos.index)
ab_neg = ab_neg.reindex(sorted(ab_neg.columns), axis=1)
ab_pos = ab_pos.reindex(sorted(ab_pos.columns), axis=1)

neg1_alpha, neg1_beta, neg1_results = fama(ab_neg, -1)
neg2_alpha, neg2_beta, neg2_results = fama(ab_neg, -2)
neg3_alpha, neg3_beta, neg3_results = fama(ab_neg, -3)
pos1_alpha, pos1_beta, pos1_results = fama(ab_pos, -1)
pos2_alpha, pos2_beta, pos2_results = fama(ab_pos, -2)
pos3_alpha, pos3_beta, pos3_results = fama(ab_pos, -3)

In [5]:
d_beta = {'ABNR 1mo' : neg1_beta, 'ABNR 2mo' : neg2_beta, 'ABNR 3mo' : neg3_beta, 
          'ABPR 1mo' : pos1_beta, 'ABPR 2mo' : pos2_beta, 'ABPR 3mo' : pos3_beta}
d_alpha = {'ABNR 1mo' : neg1_alpha, 'ABNR 2mo' : neg2_alpha, 'ABNR 3mo' : neg3_alpha, 
          'ABPR 1mo' : pos1_alpha, 'ABPR 2mo' : pos2_alpha, 'ABPR 3mo' : pos3_alpha}

betas = np.mean(pd.DataFrame(data = d_beta))
alphas = np.mean(pd.DataFrame(data = d_alpha))

In [6]:
results_no_control = pd.concat([alphas.rename('Intercept'), betas.rename('Beta')], axis = 1)
results_no_control.style.set_caption("Fama Macbeth without Control Variables")

,Intercept,Beta
ABNR 1mo,0.046513,0.147757
ABNR 2mo,0.048533,0.109448
ABNR 3mo,0.046977,0.139900
ABPR 1mo,0.061355,0.148461
ABPR 2mo,0.062203,0.137457
ABPR 3mo,0.060225,0.161317


In [7]:
results_no_control

,Intercept,Beta
ABNR 1mo,0.046513,0.147757
ABNR 2mo,0.048533,0.109448
ABNR 3mo,0.046977,0.139900
ABPR 1mo,0.061355,0.148461
ABPR 2mo,0.062203,0.137457
ABPR 3mo,0.060225,0.161317


In [8]:
#ret_co_m = pd.read_csv('Final Variables/ret_co_m.csv', index_col=0)
#ret_oc_m = pd.read_csv('Final Variables/ret_oc_m.csv', index_col=0)
size = pd.read_csv('Final Variables/monthly_size.csv')
bm = pd.read_csv('Final Variables/log_monthly_bm.csv')
ret_6m = pd.read_csv('Final Variables/monthly_cum_sum.csv')
gpa = pd.read_csv('Final Variables/annual_gross_profit.csv')
#asset_growth = pd.read_csv('Final Variables/monthly_asset_growth.csv', index_col=0)
#turnover = pd.read_csv('Final Variables/monthly_turnover.csv', index_col=0)
#illiq = pd.read_csv('Final Variables/illiquidity.csv', index_col=0)

def correct_format(var):
    var['Unnamed: 0'] = pd.to_datetime(pd.to_datetime(var['Unnamed: 0']).dt.strftime('%Y-%m'))
    var = var.set_index('Unnamed: 0')
    var = var.reindex(sorted(var.columns), axis=1)
    return var

#ret_co_m = correct_format(ret_co_m)
#ret_oc_m = correct_format(ret_oc_m)
size = correct_format(size)
bm = correct_format(bm)
ret_6m = correct_format(ret_6m)
gpa = correct_format(gpa)
#asset_growth = correct_format(asset_growth)
#turnover = correct_format(turnover)
#illiq = correct_format(illiq)

In [9]:
controls = []

for ticker in size:
    data = pd.DataFrame()
    #data['close to open'] = ret_co_m[ticker]
    #data['open to close'] = ret_oc_m[ticker]
    data['size'] = size[ticker]
    data['bm'] = bm[ticker]
    data['ret_6m'] = ret_6m[ticker]
    data['gpa'] = gpa[ticker]
    #data['asset_growth'] = asset_growth[ticker]
    #data['turnover'] = turnover[ticker]
    #data['illiq'] = illiq[ticker]
    controls.append(data)

In [10]:
neg1_alpha_c, neg1_beta_c, neg1_results_c = fama(ab_neg, -1, controls)
neg2_alpha_c, neg2_beta_c, neg2_results_c = fama(ab_neg, -2, controls)
neg3_alpha_c, neg3_beta_c, neg3_results_c = fama(ab_neg, -3, controls)
pos1_alpha_c, pos1_beta_c, pos1_results_c = fama(ab_pos, -1, controls)
pos2_alpha_c, pos2_beta_c, pos2_results_c = fama(ab_pos, -2, controls)
pos3_alpha_c, pos3_beta_c, pos3_results_c = fama(ab_pos, -3, controls)

In [11]:
d_beta = {'ABNR 1mo' : neg1_beta_c, 'ABNR 2mo' : neg2_beta_c, 'ABNR 3mo' : neg3_beta_c, 
          'ABPR 1mo' : pos1_beta_c, 'ABPR 2mo' : pos2_beta_c, 'ABPR 3mo' : pos3_beta_c}
d_alpha = {'ABNR 1mo' : neg1_alpha_c, 'ABNR 2mo' : neg2_alpha_c, 'ABNR 3mo' : neg3_alpha_c, 
          'ABPR 1mo' : pos1_alpha_c, 'ABPR 2mo' : pos2_alpha_c, 'ABPR 3mo' : pos3_alpha_c}

betas = np.mean(pd.DataFrame(data = d_beta))
alphas = np.mean(pd.DataFrame(data = d_alpha))

In [12]:
results_w_control = pd.concat([alphas.rename('Intercept'), betas.rename('Beta')], axis = 1)
results_w_control.style.set_caption("Fama Macbeth with Control Variables")

,Intercept,Beta
ABNR 1mo,0.047209,0.063135
ABNR 2mo,0.048224,0.031174
ABNR 3mo,0.045668,0.060388
ABPR 1mo,0.054155,0.038489
ABPR 2mo,0.057565,0.034362
ABPR 3mo,0.056497,0.049369
